In [25]:
import file_tagger
import training_data_generator
import tensorflow as tf
import numpy as np

from keras import layers, Sequential, models

In [17]:
DATA_DIRECTORY = "./all_respeck"
PHYSICAL_ACTIVITIES = {
    "ascending_stairs&normal_breathing",
    "shuffle_walking&normal_breathing",
    "walking&normal_breathing",
    "descending_stairs&normal_breathing",
    "misc_movements&normal_breathing",
    "running&normal_breathing",
}

In [18]:
def getLabel(recordingType):
    # print(recordingType)
    return 0 if recordingType in PHYSICAL_ACTIVITIES else 1


def generateTrainingData(directory, sequenceLength, overlap):
    tagged_data = []

    csv_dictionary = file_tagger.tag_directory(directory)

    for recordingType in csv_dictionary:
        for csv_file in csv_dictionary[recordingType]:
            sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequenceLength, overlap)
            for sequence in sequences:
                tagged_data.append((getLabel(recordingType), sequence))
            

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")    
    return tagged_data

In [26]:
trainingData = generateTrainingData(DATA_DIRECTORY, 3, 0)
# Combine all sequences and labels
sequences = [sequence for _, sequence in trainingData]
labels = [label for label, _ in trainingData]


# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
labels = np.array(labels)

there are 20160 tagged sequences in the dataset


In [23]:
def train_model_CNN(input_data, labels, unique_labels, epochs, batch_size):
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(75, 3)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels, epochs=epochs, batch_size=batch_size)

    return model

In [24]:
model = train_model_CNN(trainingData, labels, [0,1], 30, 5)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (12064, 2) + inhomogeneous part.